In [64]:
# Import Library
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [65]:
# Import file csv
return_montly = pd.read_csv(r'FF_ReturnMonthly.csv',index_col=0)
bookvalue = pd.read_csv(r'FF_BookValue.csv',index_col=0)
marketvalue = pd.read_csv(r'FF_MarketValue.csv',index_col=0)

# Re-type DataFrame index into datetime
return_montly.index=return_montly.index.astype('datetime64[ns]')
marketvalue.index=marketvalue.index.astype('datetime64[ns]')
bookvalue.index=bookvalue.index.astype('datetime64[ns]')

In [66]:
#  Checking same days between two DataFrames
def check_time(df_1, df_2):
    var =  (df_1.index == df_2.index)
    check = 0
    for i in var:
        if i == False:
            check += 1
    if check > 0:
        return False
    return True

In [67]:
check_time(bookvalue, marketvalue)

True

In [68]:
check_time(bookvalue, return_montly)

True

In [69]:
check_time(marketvalue, return_montly)

True

# Caculating BE/ME

In [70]:
# Calculating index BE/ME
BE_ME = pd.DataFrame([],index=marketvalue.index,columns=marketvalue.columns)
BE_ME.iloc[0] = bookvalue.iloc[0]/marketvalue.iloc[0]
for i in range(1,len(marketvalue.index)):
    BE_ME.iloc[i] = bookvalue.iloc[i-1]/marketvalue.iloc[i]


In [71]:
BE_ME

,VCB,VIC,VHM,HPG,ACB,BID,BVH,FPT,HDB,BCM,...,VSH,VIX,VND,HMC,VRC,MCG,VSI,YEG,YBM,BGM
2009-01-30,NaN,0.000116,NaN,0.000222,0.000221,NaN,NaN,0.000559,NaN,NaN,...,0.002664,NaN,NaN,0.057263,NaN,NaN,NaN,NaN,NaN,NaN
2009-02-27,NaN,0.000217,NaN,0.000269,0.000248,NaN,NaN,0.000658,NaN,NaN,...,0.002993,NaN,NaN,0.073711,NaN,NaN,NaN,NaN,NaN,NaN
2009-03-31,NaN,0.00023,NaN,0.000223,0.000214,NaN,NaN,0.000629,NaN,NaN,...,0.00259,NaN,NaN,0.069988,NaN,NaN,NaN,NaN,NaN,NaN
2009-04-30,NaN,0.000231,NaN,0.00017,0.000169,NaN,NaN,0.000509,NaN,NaN,...,0.002512,NaN,NaN,0.064155,NaN,NaN,NaN,NaN,NaN,NaN
2009-05-29,NaN,0.000138,NaN,0.000141,0.000146,NaN,NaN,0.000428,NaN,NaN,...,0.002471,NaN,NaN,0.047457,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,0.000082,0.000071,0.000078,0.000162,0.000372,0.000115,0.000728,0.00045,0.000422,0.000445,...,0.004321,0.006473,0.007289,0.062394,0.062883,0.04865,0.059151,0.018048,0.200291,NaN
2020-09-30,0.000081,0.00007,0.000081,0.000151,0.000351,0.000116,0.000746,0.000446,0.000382,0.000369,...,0.003335,0.005602,0.006506,0.057788,0.053912,0.051211,0.062716,0.019968,0.194715,NaN
2020-10-30,0.000082,0.00006,0.00008,0.00013,0.000327,0.000123,0.000719,0.000433,0.00038,0.000378,...,0.003784,0.004866,0.005935,0.058975,0.063292,0.053397,0.064665,0.019717,0.190732,NaN
2020-11-30,0.000073,0.000061,0.000074,0.000112,0.00029,0.000113,0.00066,0.000402,0.000329,0.000341,...,0.003864,0.004115,0.004813,0.057788,0.056824,0.04865,0.06713,0.020074,0.171269,NaN


In [72]:
# Calculating mean of market value
df_mean = pd.DataFrame([],index=marketvalue.index,columns=['Mean'])
for i in range(len(marketvalue.index)):
    df_mean.iloc[i] = marketvalue.iloc[i].mean()

threshold = df_mean.mean()[0]

# Function

In [73]:
# Function of classify scale
def classify_size(i): # i: iloc of the month you want to retrieve
    big_size = pd.DataFrame({})
    small_size = pd.DataFrame({})

    # Put variable
    month = pd.DataFrame(marketvalue.iloc[i])
    name = month.columns.astype('str')[0]  # retrieve month's name

    # Comparison function for large-small scale classifier
    # 1. Stocks with market value less than mean and not NaN
    small_size = small_size.append(month[(month[name] <= threshold) & (month[name] != np.nan)]) 
    # 2. Stocks with market value greater than mean and not NaN
    big_size   = big_size.append(month[(month[name] > threshold) & (month[name] != np.nan)]) 
    
    return small_size, big_size

# Apply function


In [74]:
fama_table = pd.DataFrame({}, columns = ['BL','BN','BH','SL','SN','SH','SMB','HML'], index = marketvalue.index)

In [75]:
for i in range(len(return_montly)):  
    # Big scale 
    month_B = pd.concat([return_montly.iloc[i],BE_ME.iloc[i],classify_size(i)[1]],axis=1)
    month_B = month_B.dropna()
    month_B.columns = ['Return_Monthly','BV/MV','Size']

    # Classify L-N-H
    size_b = np.round(len(month_B.index),0)
    b_low = int(np.round(size_b*0.3) + 1)
    b_neutral = int(b_low + np.round(size_b*0.4) + 1)
    b_high = int(b_neutral + np.round(size_b*0.3) + 1)

    month_B.sort_values(by='BV/MV',inplace=True)
    
    # The BL, BN, BH indicators
    df_BL = month_B.iloc[:b_low,:]
    df_BN = month_B.iloc[b_low:b_neutral,:]
    df_BH = month_B.iloc[b_neutral:b_high,:]

    # Small scale 
    month_S = pd.concat([return_montly.iloc[i], BE_ME.iloc[i],classify_size(i)[0]],axis=1)
    month_S = month_S.dropna()
    month_S.columns = ['Return_Monthly','BV/MV','Size']

    # Classify L-N-H
    size_s = np.round(len(month_S.index),0)
    s_low = int(np.round(size_s*0.3) + 1)
    s_neutral = int(s_low + np.round(size_s*0.4) + 1)
    s_high = int(s_neutral + np.round(size_s*0.3) + 1)

    month_S.sort_values(by='BV/MV',inplace=True)

    # The SL, SN, SH indicators
    df_SL = month_S.iloc[:s_low,:]
    df_SN = month_S.iloc[s_low:s_neutral,:]
    df_SH = month_S.iloc[s_neutral:s_high,:]

    # Calculating factors
    SL, SN, SH = df_SL.mean()[0], df_SN.mean()[0], df_SH.mean()[0]
    BL, BN, BH = df_BL.mean()[0], df_BN.mean()[0], df_BH.mean()[0]

    # Calculating SMB, HML
    SMB = (1/3*(SH + SN + SL)) - (1/3*(BH + BN + BL))
    HML = (1/2*(SH + BH)) - (1/2*(BL + SL))

    # Putting into DataFrame
    fama_table.iloc[i] = [BL,BN,BH,SL,SN,SH,SMB,HML]


In [76]:
fama_table

,BL,BN,BH,SL,SN,SH,SMB,HML
2009-01-30,-0.00341,-0.048046,-0.119705,-0.024008,-0.012041,-0.016938,0.039391,-0.054612
2009-02-27,-0.210459,-0.172973,0.0,-0.19037,-0.175192,-0.117107,-0.033079,0.141861
2009-03-31,0.090558,0.075137,0.506288,0.224033,0.145352,0.134424,-0.056058,0.16306
2009-04-30,0.178747,0.19979,0.12285,0.16318,0.174096,0.113179,-0.016977,-0.052949
2009-05-29,0.252483,0.273448,0.426613,0.384013,0.375832,0.340008,0.049103,0.065062
...,...,...,...,...,...,...,...,...
2020-08-31,0.137452,0.128984,0.142801,0.130993,0.161077,0.106033,-0.003711,-0.009805
2020-09-30,0.035599,0.114189,0.042254,0.054365,0.019256,0.013987,-0.034811,-0.016861
2020-10-30,0.041166,0.004272,-0.009762,-0.006577,-0.020239,-0.012843,-0.025112,-0.028597
2020-11-30,0.096437,0.107156,0.094356,0.127774,0.082023,0.007616,-0.026846,-0.061119


# Running model

In [77]:
gv_bond = pd.read_excel(r'Bond Yiled.xlsx',index_col=0, parse_dates=True)
return_marketvalue = pd.read_csv(r'FF_ReturnMarketvalue.csv',index_col=0, parse_dates=True)

In [78]:
# Market value return
return_marketvalue_mean = pd.DataFrame(return_marketvalue.mean(axis=1),columns=['Return_Market'])

# Government bond return (risk free return)
gv_bond.dropna(inplace=True)
gv_bond = gv_bond.pct_change()
gv_bond = gv_bond.loc['2009-01-30':'2020-12-31']

In [79]:
gv_bond.isnull().sum()

VIGBOND.    0
dtype: int64

In [80]:
# The indicators
df_ff = pd.concat([return_marketvalue_mean,gv_bond,fama_table['SMB'],fama_table['HML']],axis=1)
df_ff  = df_ff.astype('float64')

In [81]:
# Caculating Excess Return
df_ff['Excess_return'] = df_ff['Return_Market'] - df_ff['VIGBOND.']

In [82]:
df_ff

,Return_Market,VIGBOND.,SMB,HML,Excess_return
2009-01-30,-0.018892,0.000000,0.039391,-0.054612,-0.018892
2009-02-27,-0.163176,0.000000,-0.033079,0.141861,-0.163176
2009-03-31,0.197017,-0.071429,-0.056058,0.163060,0.268445
2009-04-30,0.157784,-0.010989,-0.016977,-0.052949,0.168773
2009-05-29,0.359573,0.022222,0.049103,0.065062,0.337350
...,...,...,...,...,...
2020-08-31,0.134950,-0.031355,-0.003711,-0.009805,0.166306
2020-09-30,0.038182,-0.045249,-0.034811,-0.016861,0.083431
2020-10-30,-0.008197,-0.061611,-0.025112,-0.028597,0.053415
2020-11-30,0.081619,-0.059441,-0.026846,-0.061119,0.141060


In [83]:
# Fama-French model
model = 'Excess_return ~ Return_Market + SMB + HML' # E(ri) - rf = alpha + B1*MKT + B2*SMB + B3*HML
ff_model = smf.ols(model, data = df_ff).fit()

# Result
print(ff_model.summary())

                            OLS Regression Results                            
Dep. Variable:          Excess_return   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.759
Method:                 Least Squares   F-statistic:                     151.1
Date:                Fri, 29 Oct 2021   Prob (F-statistic):           1.03e-43
Time:                        17:38:55   Log-Likelihood:                 241.53
No. Observations:                 144   AIC:                            -475.1
Df Residuals:                     140   BIC:                            -463.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0021      0.004      0.503